## Subset and analyze Sentinel-2 imagey
* Fetches the Sentinel imagery layer from the Sentinel Living Atlas item:  
    * https://www.arcgis.com/home/item.html?id=fd61b9e0c69c4e14bebd50a9a968348c
* Creates a geometry (here, by geocoding)
* Filters the imagery for a specific time

In [ ]:
#import packages
from arcgis import GIS
from arcgis.geocoding import geocode
from arcgis.raster import ImageryLayer, Raster
from arcgis.raster.functions import apply
from arcgis.geometry import Polygon, filters

import pandas as pd
from datetime import datetime

#instantitate the GIS object
gis = GIS('home')

In [ ]:
#Fetch the imagery service and extract as a layer
s2_item = gis.content.get('fd61b9e0c69c4e14bebd50a9a968348c')
s2_images = s2_item.layers[0]

In [ ]:
#Get its spatial reference
s2_sr = s2_images.properties.spatialReference

In [ ]:
#Print the available [NDWI] raster functions
for rasterfunc in s2_images.properties.rasterFunctionInfos:
    if 'NDWI' in rasterfunc.name:
        print(f'***{rasterfunc.name}***')
        print(rasterfunc.description,'\n')

In [ ]:
#Set the area of interest (in the spatial reference of the image)
aoi = geocode('St. Louis, MO', out_sr=s2_sr)[0]

In [ ]:
#Get NWDI from the image 
ndwi_images = apply(s2_images, 'NDWI Raw')
#Subset for the area of interest
ndwi_images.extent = aoi['extent']

In [ ]:
#Display
ndwi_images

In [ ]:
#Create the geometry - we'll use St. Louis
stl = geocode(address = "St. Louis, MO", as_featureset=True , out_sr = s2_sr).sdf
stl_geom = stl.spatial.bbox
stl_filter = filters.intersects(stl_geom)

In [ ]:
the_map = gis.map('St. Louis, MO',zoomlevel=12)
the_map

In [ ]:
#Filter the Sentintal imagery layer for date and location
s2_images_filtered = s2_images.filter_by(
    geometry=filters.intersects(the_map.extent),
    time=[datetime(2024,11,8),datetime(2024,11,14)]
)

In [ ]:
#Get the list of images as a dataframe
df_imgs = s2_images_filtered.query().sdf
#Sort on cloud cover
df_imgs.sort_values(by='cloudcover',inplace=True)
df_imgs.head()

In [ ]:
#Get the name of the first(best) image
the_name = df_imgs.iloc[0,1]
the_name

In [ ]:
the_image = s2_images.filter_by(
    where=f"name = '{the_name}'"
)
the_image = apply(the_image,'NDWI Raw')
the_image.extent=the_map.extent
the_image

In [ ]:
#Export the image
the_tiff = the_image.export_image(
    export_format = 'tiff',
    save_folder=',',
    save_file='foo.tiff'
)

In [ ]:
import requests
save_path = 'the_image.tif'
response = requests.get(the_tiff['href'])
if response.status_code == 200:
    with open(save_path, 'wb') as file:
        file.write(response.content)
    print(f"Image saved successfully at: {save_path}")
else:
    print("Failed to download the image")